In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import torch

import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset

##Carga del dataset
Dataset: https://huggingface.co/datasets/Emilianohack6950/Wikipedia-es

In [2]:
#Cargamos el dataset
ds = load_dataset("Emilianohack6950/Wikipedia-es")

#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("shubhammaindola/harry-potter-books")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [3]:
import os

# The 'path' variable is a directory, not a file.
# We need to list its contents to find the actual data file.
"""
print(f"Contents of the downloaded directory '{path}':")
for filename in os.listdir(path):
    print(filename)

# Assuming the dataset contains a single JSON file, we'll try to load the first one we find.
json_files = [f for f in os.listdir(path) if f.endswith('.json')]

if json_files:
    data_file_path = os.path.join(path, json_files[0])
    print(f"\nAttempting to load data from: {data_file_path}")
    with open(data_file_path, "r") as file:
        data = json.load(file)
    display(data)
else:
    print("No JSON files found in the downloaded directory. Please specify the correct file.")
    """

'\nprint(f"Contents of the downloaded directory \'{path}\':")\nfor filename in os.listdir(path):\n    print(filename)\n\n# Assuming the dataset contains a single JSON file, we\'ll try to load the first one we find.\njson_files = [f for f in os.listdir(path) if f.endswith(\'.json\')]\n\nif json_files:\n    data_file_path = os.path.join(path, json_files[0])\n    print(f"\nAttempting to load data from: {data_file_path}")\n    with open(data_file_path, "r") as file:\n        data = json.load(file)\n    display(data)\nelse:\n    print("No JSON files found in the downloaded directory. Please specify the correct file.")\n    '

In [4]:
"""
with open(path + "/02 Harry Potter and the Chamber of Secrets.txt", "r") as file:
  content = file.read()
"""
#len(content)
train_contenido = [item["contenido"] for item in ds["train"]]


In [5]:
train_contenido = [item["contenido"] for item in ds["train"]]
test_contenido = [item["contenido"] for item in ds["test"]]

todo_contenido = train_contenido + test_contenido

# Unir todo en una sola cadena
texto_total = " ".join(todo_contenido)

# Contar signos de puntuación
cant_comas = texto_total.count(',')
cant_puntos = texto_total.count('.')
cant_preg_abre = texto_total.count('¿')
cant_preg_cierra = texto_total.count('?')

# Contar letras mayúsculas (A-Z, incluyendo acentos)
cant_mayus = len(re.findall(r'[A-ZÁÉÍÓÚÜÑ]', texto_total))

print("Estadísticas del dataset:")
print(f"Comas (,): {cant_comas:,}")
print(f"Puntos (.): {cant_puntos:,}")
print(f"Signos de pregunta abiertos (¿): {cant_preg_abre:,}")
print(f"Signos de pregunta cerrados (?): {cant_preg_cierra:,}")
print(f"Letras mayúsculas: {cant_mayus:,}")

Estadísticas del dataset:
Comas (,): 22,160
Puntos (.): 16,364
Signos de pregunta abiertos (¿): 21
Signos de pregunta cerrados (?): 53
Letras mayúsculas: 72,640


## Funciones de preprocesamiento

In [6]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

# Funcion provista por la catedra
def get_multilingual_token_embedding(token: str):
  """     Devuelve el embedding (estático) para el token.     """
  if not isinstance(token, str):
        print(f" ⚠️ Token recibido no es string: {token} — convertido a string.")
        token = str(token)

    # Obtener ID desde tokenizer
  token_id = tokenizer.convert_tokens_to_ids(token)

  # Caso OOV: token no existe → usamos UNK pero mantenemos el print original
  if token_id is None or token_id == tokenizer.unk_token_id:
      print(f" ❌ El token '{token}' no pertenece al vocabulario de multilingual BERT. Usando [UNK].")
      token_id = tokenizer.unk_token_id
  else:
      print(f" ✅ Token: '{token}' | ID: {token_id}")

  # Seguridad extra: forzar ID dentro del rango del vocab
  vocab_size = model.embeddings.word_embeddings.weight.shape[0]
  if not (0 <= token_id < vocab_size):
      print(f" ⚠️ token_id fuera de rango ({token_id}) → usando [UNK]")
      token_id = tokenizer.unk_token_id

  # Devuelve SIEMPRE un tensor válido
  embedding_vector = model.embeddings.word_embeddings.weight[token_id]

  print(f"Embedding shape: {embedding_vector.shape}")
  return embedding_vector


def tipo_capitalizacion(palabra):
    conteo_mayusculas = 0
    for caracter in palabra:
        if caracter.isupper():
            conteo_mayusculas += 1

    if palabra.isupper():
        return 3
    else:
        if conteo_mayusculas >= 2:
            return 2
        else:
            if conteo_mayusculas == 1:
                if palabra[0].isupper():
                    return 1
                else:
                    return 2
            else:
                return 0

#Capaz conviene tokenizar todo de una en vez de palabra por palabra

def tokenizar_y_etiquetar(texto_original, instancia_id = 1):
  SIGNOS_ESPANOL = "¿¡?.,!\"'’“”()[]:;—-"

  #Separamos por palabra y signo
  partes = re.findall(r"\w+['’]?\w*|[¿?.,]", texto_original)
  resultado = []

  #Para cada Palabra y Signo, aplanamos y tokenizamos
  #Extraemos Capitalización y puntuación de la palabra y signos adyacentes, y se la asignamos a los tokens
  for i in range(len(partes)):
    parte = partes[i]

    if(parte not in  SIGNOS_ESPANOL):
      #tokens = tokenizer.tokenize(parte.lower().strip("¿?.,¡!"))
      tokens = tokenizer.tokenize(parte.lower()) # Sacamos mayúsculas y la tokenizamos
      #Obtenemos la capitalización
      cap = tipo_capitalizacion(parte)

      #Obtenemos la puntuación por token
      punt_ini = '¿' if partes[i-1]=='¿' else ''
      punt_fin = ''
      if(i<len(partes)-1):
        if partes[i+1] in '.?,':
          punt_fin = partes[i+1]

      for j in range(len(tokens)):
        tok = tokens[j]
        punt_ini_token = ''
        punt_fin_token = ''
        if(j==0):#Si es el primer token se le agrega puntuación inicial ¿
            punt_ini_token = punt_ini
        if (j==len(tokens)-1): #Si el último token se le agrega puntuación final
            punt_fin_token = punt_fin

        token_id = tokenizer.convert_tokens_to_ids(tok)
        resultado.append({
                    "instancia_id": instancia_id,
                    "token_id": token_id,
                    "token": tok,
                    "punt_inicial": punt_ini_token,
                    "punt_final": punt_fin_token,
                    "capitalizacion": cap
                })

  return resultado

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

## Testing para tokenizar y etiquetar

In [7]:
import unittest

class TestFunciones(unittest.TestCase):

    def simplificar_salida(self, salida):
        return [
            [item["token"], item["punt_inicial"], item["punt_final"], item["capitalizacion"]]
            for item in salida
        ]

    def test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra(self):
        texto_1 = "Los árboles altos"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['los','','',1], ['árboles','','',0], ['altos','','',0]]
        )

    def test02_reconoce_Punto_final_mas_de_una_palabra(self):
        texto_1 = "Los árboles altos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['los','','',1], ['árboles','','',0], ['altos','','.',0]]
        )

    def test03_reconoce_Capitalizacion_Todos_los_tokens(self):
        texto_1 = "Hermosos son los árboles altos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['her','','',1], ['##mos','','',1], ['##os','','',1],
             ['son','','',0], ['los','','',0], ['árboles','','',0], ['altos','','.',0]]
        )

    def test04_reconoce_Capitalizacion_todos_los_tokens_May_Min(self):
        texto_1 = "Hermosos son los árboles hermosos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['her','','',1], ['##mos','','',1], ['##os','','',1],
             ['son','','',0], ['los','','',0], ['árboles','','',0],
             ['her','','',0], ['##mos','','',0], ['##os','','.',0]]
        )

    def test05_reconoce_capitalizacion_Signo_pregunta_punto(self):
        texto_1 = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['cu','¿','',1], ['##ánd','','',1], ['##o','','',1], ['va','','',0],
             ['##mos','','',0], ['a','','',0], ['m','','',2], ['##c','','',2],
             ['##dona','','',2], ['##ld','','',2], ["'",'','',2], ['s','','?',2],
             ['ellos','','',1], ['no','','',0], ['viene','','',0], ['##n','','',0],
             ['hoy','','.',0], ['dó','¿','',1], ['##nde','','',1],
             ['están','','',0], ['ahora','','?',0]]
        )

    def test06_reconoce_comas(self):
        texto_1 = "Si, es correcto."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['si','',',',1], ['es','','',0], ['correct','','',0], ['##o','','.',0]]
        )

    def test07_reconoce_varias_comas(self):
        texto = "Sí, claro, entiendo."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['sí','',',',1], ['claro','',',',0], ['ent','','',0], ['##iendo','','.',0]]
        )

    def test08_reconoce_todas_mayusculas(self):
        texto = "Me gustaría ir a la NASA."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['me','','',1], ['gu','','',0], ['##star','','',0], ['##ía','','',0],
             ['ir','','',0], ['a','','',0], ['la','','',0], ['nasa','','.',3]]
        )

    def test09_reconoce_algunas_mayusculas(self):
        texto = "El iPhone es caro."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['el','','',1], ['i','','',2], ['##phone','','',2], ['es','','',0],
             ['car','','',0], ['##o','','.',0]]
        )

    def test10_otros_signos_no_afectan(self):
        texto = "¡Hola Mundo!(a)[A];a:A-a"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['hol','','',1], ['##a','','',1], ['mundo','','',1],
             ['a','','',0], ['a','','',3], ['a','','',0],
             ['a','','',3], ['a','','',0]]
        )


if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)

test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra (__main__.TestFunciones.test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra) ... ok
test02_reconoce_Punto_final_mas_de_una_palabra (__main__.TestFunciones.test02_reconoce_Punto_final_mas_de_una_palabra) ... ok
test03_reconoce_Capitalizacion_Todos_los_tokens (__main__.TestFunciones.test03_reconoce_Capitalizacion_Todos_los_tokens) ... ok
test04_reconoce_Capitalizacion_todos_los_tokens_May_Min (__main__.TestFunciones.test04_reconoce_Capitalizacion_todos_los_tokens_May_Min) ... ok
test05_reconoce_capitalizacion_Signo_pregunta_punto (__main__.TestFunciones.test05_reconoce_capitalizacion_Signo_pregunta_punto) ... ok
test06_reconoce_comas (__main__.TestFunciones.test06_reconoce_comas) ... ok
test07_reconoce_varias_comas (__main__.TestFunciones.test07_reconoce_varias_comas) ... ok
test08_reconoce_todas_mayusculas (__main__.TestFunciones.test08_reconoce_todas_mayusculas) ... ok
test09_reconoce_algunas_mayusculas (__main

In [8]:
texto_1 = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
texto_2 = "cuándo vamos a mcdonald's ellos no vienen hoy dónde están ahora"
ej1_tokens = tokenizer.tokenize(texto_1)
ej2_tokens = tokenizer.tokenize(texto_2)
print(f"Tokens original: {ej1_tokens}")
print(f"Tokens plano: {ej2_tokens}")



Tokens original: ['¿', 'Cu', '##ánd', '##o', 'va', '##mos', 'a', 'McDonald', "'", 's', '?', 'El', '##los', 'no', 'viene', '##n', 'hoy', '.', '¿', 'D', '##ón', '##de', 'están', 'ahora', '?']
Tokens plano: ['cu', '##ánd', '##o', 'va', '##mos', 'a', 'm', '##c', '##dona', '##ld', "'", 's', 'ellos', 'no', 'viene', '##n', 'hoy', 'dó', '##nde', 'están', 'ahora']


## Conversion en embeddings y tensores.

In [9]:
MAP_PUNT_INI = {"": 0, "¿": 1}
MAP_PUNT_FIN = {"": 0, ",": 1, ".": 2, "?": 3}

# Faltaría usar dataset y data loader de pytorch

def procesar_texto_a_tensores(texto, instancia_id=1):
    etiquetas = tokenizar_y_etiquetar(texto, instancia_id)

    tokens = [fila["token"] for fila in etiquetas]

    lista_embeddings = []
    for tok in tokens:
        emb = get_multilingual_token_embedding(tok)
        lista_embeddings.append(emb)

    embeddings_tensor = torch.stack(lista_embeddings)

    # Outputs (convertidos a tensores)
    y_cap = torch.tensor([fila["capitalizacion"] for fila in etiquetas])
    y_punt_ini = torch.tensor([MAP_PUNT_INI[fila["punt_inicial"]] for fila in etiquetas])
    y_punt_fin = torch.tensor([MAP_PUNT_FIN[fila["punt_final"]] for fila in etiquetas])


    return {
        "instancia_id": instancia_id,
        "tokens": tokens,
        "X_emb": embeddings_tensor,
        "y_cap": y_cap,
        "y_punt_ini": y_punt_ini,
        "y_punt_fin": y_punt_fin,
        "etiquetas": etiquetas
    }

In [10]:
texto = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
datos = procesar_texto_a_tensores(texto, instancia_id=1)

print("Tokens:", datos["tokens"])
print("Embeddings shape:", datos["X_emb"].shape)
print("Capitalización:", datos["y_cap"])
print("Puntuación inicial:", datos["y_punt_ini"])
print("Puntuación final:", datos["y_punt_fin"])

 ✅ Token: 'cu' | ID: 10854
Embedding shape: torch.Size([768])
 ✅ Token: '##ánd' | ID: 101439
Embedding shape: torch.Size([768])
 ✅ Token: '##o' | ID: 10133
Embedding shape: torch.Size([768])
 ✅ Token: 'va' | ID: 10321
Embedding shape: torch.Size([768])
 ✅ Token: '##mos' | ID: 13386
Embedding shape: torch.Size([768])
 ✅ Token: 'a' | ID: 169
Embedding shape: torch.Size([768])
 ✅ Token: 'm' | ID: 181
Embedding shape: torch.Size([768])
 ✅ Token: '##c' | ID: 10350
Embedding shape: torch.Size([768])
 ✅ Token: '##dona' | ID: 64674
Embedding shape: torch.Size([768])
 ✅ Token: '##ld' | ID: 12620
Embedding shape: torch.Size([768])
 ✅ Token: ''' | ID: 112
Embedding shape: torch.Size([768])
 ✅ Token: 's' | ID: 187
Embedding shape: torch.Size([768])
 ✅ Token: 'ellos' | ID: 16169
Embedding shape: torch.Size([768])
 ✅ Token: 'no' | ID: 10192
Embedding shape: torch.Size([768])
 ✅ Token: 'viene' | ID: 12266
Embedding shape: torch.Size([768])
 ✅ Token: '##n' | ID: 10115
Embedding shape: torch.Size([768]

# Random Forest

Los features que me parecen útiles para este problema son: la posición de la palabra en la oración, la distancia entre el embedding previo y el posterior.

In [11]:
texto = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
datos = procesar_texto_a_tensores(texto, instancia_id=1)

 ✅ Token: 'cu' | ID: 10854
Embedding shape: torch.Size([768])
 ✅ Token: '##ánd' | ID: 101439
Embedding shape: torch.Size([768])
 ✅ Token: '##o' | ID: 10133
Embedding shape: torch.Size([768])
 ✅ Token: 'va' | ID: 10321
Embedding shape: torch.Size([768])
 ✅ Token: '##mos' | ID: 13386
Embedding shape: torch.Size([768])
 ✅ Token: 'a' | ID: 169
Embedding shape: torch.Size([768])
 ✅ Token: 'm' | ID: 181
Embedding shape: torch.Size([768])
 ✅ Token: '##c' | ID: 10350
Embedding shape: torch.Size([768])
 ✅ Token: '##dona' | ID: 64674
Embedding shape: torch.Size([768])
 ✅ Token: '##ld' | ID: 12620
Embedding shape: torch.Size([768])
 ✅ Token: ''' | ID: 112
Embedding shape: torch.Size([768])
 ✅ Token: 's' | ID: 187
Embedding shape: torch.Size([768])
 ✅ Token: 'ellos' | ID: 16169
Embedding shape: torch.Size([768])
 ✅ Token: 'no' | ID: 10192
Embedding shape: torch.Size([768])
 ✅ Token: 'viene' | ID: 12266
Embedding shape: torch.Size([768])
 ✅ Token: '##n' | ID: 10115
Embedding shape: torch.Size([768]

###Extraer posición

In [12]:
def agregar_position(lista_tensores):
    """
    Modifico los tokens, agrego un campo 'position' con respecto a toda la instancia.
    """
    for idx, fila in enumerate(lista_tensores["etiquetas"]):
        fila["position"] = idx
    return lista_tensores

agregar_position(datos)

{'instancia_id': 1,
 'tokens': ['cu',
  '##ánd',
  '##o',
  'va',
  '##mos',
  'a',
  'm',
  '##c',
  '##dona',
  '##ld',
  "'",
  's',
  'ellos',
  'no',
  'viene',
  '##n',
  'hoy',
  'dó',
  '##nde',
  'están',
  'ahora'],
 'X_emb': tensor([[-0.0359, -0.0098,  0.0428,  ..., -0.0560, -0.0796, -0.0400],
         [-0.0786, -0.0631,  0.0020,  ...,  0.0548, -0.0627, -0.0374],
         [ 0.0088, -0.0353, -0.0401,  ..., -0.0407,  0.0383,  0.0374],
         ...,
         [-0.0155, -0.0555,  0.0295,  ...,  0.0256, -0.0288, -0.0430],
         [-0.0094, -0.0267, -0.0801,  ..., -0.0423, -0.0967,  0.0284],
         [ 0.0621, -0.0263, -0.0598,  ...,  0.0179, -0.0331,  0.0587]],
        grad_fn=<StackBackward0>),
 'y_cap': tensor([1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0]),
 'y_punt_ini': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 'y_punt_fin': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 3]),
 'etiquetas': [{'instancia_id'

###Obtener distancia previo

In [13]:
def agregar_dist_prev(lista_tokens):
    """
    Modifico los tokens, añado la distancia con el token previo.
    """
    emb = lista_tokens["X_emb"]
    num_tokens = emb.size(0)

    dist_prev = [0.0]  # El primer token NO tiene previo

    for i in range(1, num_tokens):
        dist = torch.norm(emb[i] - emb[i-1]).item()
        dist_prev.append(dist)

    # Guardamos las distancias en etiquetas:
    for i, fila in enumerate(lista_tokens["etiquetas"]):
        fila["dist_prev"] = dist_prev[i]

    return lista_tokens

agregar_dist_prev(datos)

{'instancia_id': 1,
 'tokens': ['cu',
  '##ánd',
  '##o',
  'va',
  '##mos',
  'a',
  'm',
  '##c',
  '##dona',
  '##ld',
  "'",
  's',
  'ellos',
  'no',
  'viene',
  '##n',
  'hoy',
  'dó',
  '##nde',
  'están',
  'ahora'],
 'X_emb': tensor([[-0.0359, -0.0098,  0.0428,  ..., -0.0560, -0.0796, -0.0400],
         [-0.0786, -0.0631,  0.0020,  ...,  0.0548, -0.0627, -0.0374],
         [ 0.0088, -0.0353, -0.0401,  ..., -0.0407,  0.0383,  0.0374],
         ...,
         [-0.0155, -0.0555,  0.0295,  ...,  0.0256, -0.0288, -0.0430],
         [-0.0094, -0.0267, -0.0801,  ..., -0.0423, -0.0967,  0.0284],
         [ 0.0621, -0.0263, -0.0598,  ...,  0.0179, -0.0331,  0.0587]],
        grad_fn=<StackBackward0>),
 'y_cap': tensor([1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0]),
 'y_punt_ini': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 'y_punt_fin': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 3]),
 'etiquetas': [{'instancia_id'

### Obtener distancia siguiente

In [14]:
def agregar_dist_next(lista_tokens):
    emb = lista_tokens["X_emb"]
    num_tokens = emb.size(0)

    dist_next = []

    for i in range(num_tokens-1):
        dist = torch.norm(emb[i] - emb[i+1]).item()
        dist_next.append(dist)

    dist_next.append(0.0)  #El último NO tiene siguiente

    for i, fila in enumerate(lista_tokens["etiquetas"]):
        fila["dist_next"] = dist_next[i]

    return lista_tokens


agregar_dist_next(datos)

{'instancia_id': 1,
 'tokens': ['cu',
  '##ánd',
  '##o',
  'va',
  '##mos',
  'a',
  'm',
  '##c',
  '##dona',
  '##ld',
  "'",
  's',
  'ellos',
  'no',
  'viene',
  '##n',
  'hoy',
  'dó',
  '##nde',
  'están',
  'ahora'],
 'X_emb': tensor([[-0.0359, -0.0098,  0.0428,  ..., -0.0560, -0.0796, -0.0400],
         [-0.0786, -0.0631,  0.0020,  ...,  0.0548, -0.0627, -0.0374],
         [ 0.0088, -0.0353, -0.0401,  ..., -0.0407,  0.0383,  0.0374],
         ...,
         [-0.0155, -0.0555,  0.0295,  ...,  0.0256, -0.0288, -0.0430],
         [-0.0094, -0.0267, -0.0801,  ..., -0.0423, -0.0967,  0.0284],
         [ 0.0621, -0.0263, -0.0598,  ...,  0.0179, -0.0331,  0.0587]],
        grad_fn=<StackBackward0>),
 'y_cap': tensor([1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0]),
 'y_punt_ini': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 'y_punt_fin': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 3]),
 'etiquetas': [{'instancia_id'

### Unir features

In [15]:
def construir_features(lista_tokens):
    """
    devuelve dict por procesar_texto_a_tensores + funciones de features
    """
    lista_features = []

    for fila in lista_tokens["etiquetas"]:
        f = [
            fila["position"],
            fila["dist_prev"],
            fila["dist_next"]
        ]
        lista_features.append(f)

    X = torch.tensor(lista_features, dtype=torch.float32)
    return X

construir_features(datos)

tensor([[ 0.0000,  0.0000,  1.7063],
        [ 1.0000,  1.7063,  1.6893],
        [ 2.0000,  1.6893,  1.3785],
        [ 3.0000,  1.3785,  1.6255],
        [ 4.0000,  1.6255,  1.5652],
        [ 5.0000,  1.5652,  1.2556],
        [ 6.0000,  1.2556,  1.3190],
        [ 7.0000,  1.3190,  1.7552],
        [ 8.0000,  1.7552,  1.8052],
        [ 9.0000,  1.8052,  1.5537],
        [10.0000,  1.5537,  1.2158],
        [11.0000,  1.2158,  1.5683],
        [12.0000,  1.5683,  1.5848],
        [13.0000,  1.5848,  1.5979],
        [14.0000,  1.5979,  1.5616],
        [15.0000,  1.5616,  1.5972],
        [16.0000,  1.5972,  1.7939],
        [17.0000,  1.7939,  1.7417],
        [18.0000,  1.7417,  1.6002],
        [19.0000,  1.6002,  1.3168],
        [20.0000,  1.3168,  0.0000]])

In [16]:
def generar_features_posicionales(lista_tokens):
  agregar_position(lista_tokens)
  agregar_dist_prev(lista_tokens)
  agregar_dist_next(lista_tokens)
  return construir_features(lista_tokens)

In [17]:
generar_features_posicionales(datos) #última prueba

tensor([[ 0.0000,  0.0000,  1.7063],
        [ 1.0000,  1.7063,  1.6893],
        [ 2.0000,  1.6893,  1.3785],
        [ 3.0000,  1.3785,  1.6255],
        [ 4.0000,  1.6255,  1.5652],
        [ 5.0000,  1.5652,  1.2556],
        [ 6.0000,  1.2556,  1.3190],
        [ 7.0000,  1.3190,  1.7552],
        [ 8.0000,  1.7552,  1.8052],
        [ 9.0000,  1.8052,  1.5537],
        [10.0000,  1.5537,  1.2158],
        [11.0000,  1.2158,  1.5683],
        [12.0000,  1.5683,  1.5848],
        [13.0000,  1.5848,  1.5979],
        [14.0000,  1.5979,  1.5616],
        [15.0000,  1.5616,  1.5972],
        [16.0000,  1.5972,  1.7939],
        [17.0000,  1.7939,  1.7417],
        [18.0000,  1.7417,  1.6002],
        [19.0000,  1.6002,  1.3168],
        [20.0000,  1.3168,  0.0000]])

### Construir DataFrame


In [18]:
train_contenido = [item["contenido"] for item in ds["train"]]
test_contenido = [item["contenido"] for item in ds["test"]]

In [19]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd

In [20]:
def construir_dataset_RF(lista_textos):
    filas = []
    instancia_global = 0

    for texto in lista_textos:
        proc = procesar_texto_a_tensores(texto, instancia_id=instancia_global)
        instancia_global += 1

        tokens = proc["tokens"]
        X_emb = proc["X_emb"]
        y_cap = proc["y_cap"]
        y_punt_ini = proc["y_punt_ini"]
        y_punt_fin = proc["y_punt_fin"]

        num_tokens = len(tokens)

        # Convertir de tensor → numpy
        X_np = X_emb.detach().cpu().numpy()

        for i in range(num_tokens):
            fila = {}

            fila["instancia_id"] = proc["instancia_id"]
            fila["token"] = tokens[i]
            fila["position"] = i
            fila["token_len"] = len(tokens[i])

            # Norma del embedding
            emb_i = X_np[i]
            fila["emb_norm"] = float(np.linalg.norm(emb_i))

            # Distancia al anterior
            if i > 0:
                fila["dist_prev"] = float(np.linalg.norm(emb_i - X_np[i-1]))
            else:
                fila["dist_prev"] = np.nan

            # Distancia al siguiente
            if i < num_tokens - 1:
                fila["dist_next"] = float(np.linalg.norm(emb_i - X_np[i+1]))
            else:
                fila["dist_next"] = np.nan

            # Etiquetas
            fila["y_cap"] = int(y_cap[i].item())
            fila["y_punt_ini"] = int(y_punt_ini[i].item())
            fila["y_punt_fin"] = int(y_punt_fin[i].item())

            filas.append(fila)

    return pd.DataFrame(filas)

In [21]:
df_rf_train = construir_dataset_RF(train_contenido)

Se han truncado las últimas 5000 líneas del flujo de salida.
 ✅ Token: '##cu' | ID: 12352
Embedding shape: torch.Size([768])
 ✅ Token: '##bier' | ID: 66727
Embedding shape: torch.Size([768])
 ✅ Token: '##ta' | ID: 10213
Embedding shape: torch.Size([768])
 ✅ Token: 'por' | ID: 10183
Embedding shape: torch.Size([768])
 ✅ Token: 'los' | ID: 10182
Embedding shape: torch.Size([768])
 ✅ Token: 'investigadores' | ID: 78542
Embedding shape: torch.Size([768])
 ✅ Token: 'franceses' | ID: 33016
Embedding shape: torch.Size([768])
 ✅ Token: 'radi' | ID: 31031
Embedding shape: torch.Size([768])
 ✅ Token: '##cados' | ID: 64124
Embedding shape: torch.Size([768])
 ✅ Token: 'en' | ID: 10110
Embedding shape: torch.Size([768])
 ✅ Token: 'santi' | ID: 91168
Embedding shape: torch.Size([768])
 ✅ Token: '##ago' | ID: 22163
Embedding shape: torch.Size([768])
 ✅ Token: 'del' | ID: 10127
Embedding shape: torch.Size([768])
 ✅ Token: 'estero' | ID: 85157
Embedding shape: torch.Size([768])
 ✅ Token: 'em' | ID: 102

##Capitalización

In [22]:
df = df_rf_train.copy()

In [30]:
df["is_subword"] = df["token"].apply(lambda x: 1 if x.startswith("##") else 0)
df["token_clean"] = df["token"].apply(lambda x: x.lstrip("#"))

df["token_first_char"] = df["token_clean"].apply(
    lambda x: ord(x[0].lower()) if len(x) > 0 else 0
)

# Rellenar NaNs de dist_prev/dist_next
#df["dist_prev"] = df["dist_prev"].fillna(0)
#df["dist_next"] = df["dist_next"].fillna(0)

atributos = [
    "position",
    "token_len",
    "emb_norm",
    "dist_prev",
    "dist_next",
    "is_subword",
    "token_first_char",
]

In [31]:
instancias = df["instancia_id"].unique()
train_ids, test_ids = train_test_split(instancias, test_size=0.2, random_state=42)

df_train = df[df["instancia_id"].isin(train_ids)]
df_test = df[df["instancia_id"].isin(test_ids)]

X_train = df_train[atributos]
y_train = df_train["y_cap"]

X_test = df_test[atributos]
y_test = df_test["y_cap"]

In [48]:
rf_cap = RandomForestClassifier(
    n_estimators=150, #Probe con 200, tarda 5 minutos y no mejora mucho.
    max_depth=None,
    class_weight="balanced",
    random_state=0
)

rf_cap.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=150,
                       random_state=0)

In [49]:
y_pred = rf_cap.predict(X_test)

In [47]:
print("F1:", f1_score(y_test, y_pred, labels=[0,1,2,3] ,average="macro"))
print("Accuracy:", accuracy_score(y_test, y_pred))

F1: 0.5673870551158829
Accuracy: 0.8404114083410905


##Puntuación Inicial

In [52]:
df = df_rf_train.copy()

In [62]:
df["is_subword"] = df["token"].apply(lambda x: 1 if x.startswith("##") else 0)
df["token_clean"] = df["token"].apply(lambda x: x.lstrip("#"))

df["token_first_char"] = df["token_clean"].apply(
    lambda x: ord(x[0].lower()) if len(x) > 0 else 0
)

# Rellenar NaNs de dist_prev/dist_next
df["dist_prev"] = df["dist_prev"].fillna(0)
df["dist_next"] = df["dist_next"].fillna(0)

atributos = [
    "position",
    "token_len",
    "emb_norm",
    "dist_prev",
    "dist_next",
    "is_subword",
    "token_first_char"
]

X = df[atributos]
y = df["y_punt_ini"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [64]:
rf_punt_ini = RandomForestClassifier(
    n_estimators=150,
    max_depth=None,
    class_weight="balanced",
    random_state=0
)

rf_punt_ini.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=150,
                       random_state=0)

In [58]:
y_pred = rf_punt_ini.predict(X_test)

In [61]:
print("F1:", f1_score(y_test, y_pred, labels=[0, 1] ,average="macro"))
print("Accuracy:", accuracy_score(y_test, y_pred))

F1: 0.49998758997269793
Accuracy: 0.9999503611228314


##Puntuación Final

In [67]:
df = df_rf_train.copy()

In [69]:
df["is_subword"] = df["token"].apply(lambda x: 1 if x.startswith("##") else 0)
df["token_clean"] = df["token"].apply(lambda x: x.lstrip("#"))

df["token_first_char"] = df["token_clean"].apply(
    lambda x: ord(x[0].lower()) if len(x) > 0 else 0
)

# Rellenar NaNs de dist_prev/dist_next
df["dist_prev"] = df["dist_prev"].fillna(0)
df["dist_next"] = df["dist_next"].fillna(0)

atributos = [
    "position",
    "token_len",
    "emb_norm",
    "dist_prev",
    "dist_next",
    "is_subword",
    "token_first_char"
]

X = df[atributos]
y = df["y_punt_fin"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [70]:
y_pred = rf_punt_ini.predict(X_test)

In [72]:
print("F1:", f1_score(y_test, y_pred, labels=[0, 1, 2, 3] ,average="macro"))
print("Accuracy:", accuracy_score(y_test, y_pred))

F1: 0.2401469175056271
Accuracy: 0.9241642054056737
